In [19]:
!pip install chromadb ollama PyPDF2

In [20]:
import chromadb
import ollama  # used for embeddings, requires Ollama running and embedding model downloaded
import PyPDF2
import uuid
import time
import os

In [21]:
client = chromadb.PersistentClient(path="resume_db")
collection = client.get_or_create_collection("resumes")

In [22]:
import PyPDF2

pdf_path = "/Users/sakshamfaujdar/Developer/NGIL/resume_reviewer/Saksham_resume.pdf"
with open(pdf_path, "rb") as f:
    reader = PyPDF2.PdfReader(f)
    resume_text = "\n".join(page.extract_text() for page in reader.pages if page.extract_text())

print("Extracted resume text:", resume_text[:400])  # Preview

# Now safe to use in collection.add()


Extracted resume text: SAKSHAM
+91 9818260018 ⋄Delhi, India
sakshamarijay@gmail.com ⋄linkedin.com/in/saksham1211 ⋄github.com/itssaksham12
SUMMARY
AI/ML engineer with hands-on experience in Python, TensorFlow, and deep learning. Proficient in NLP, Generative
AI, and Computer Vision. Passionate about building scalable AI solutions and enhancing model efficiency.
EDUCATION
Bachelor of Technology, Computer Science and Engin


In [23]:
import os
import time
import uuid
import PyPDF2
import ollama

def add_resume_from_pdf(pdf_path, job_desc, skills_list):
    # Extract text from PDF using PyPDF2
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        resume_text = "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
    
    # Generate embedding for the resume
    emb_resp = ollama.embed(model="nomic-embed-text", input=resume_text)
    embedding = emb_resp.get("embedding") or emb_resp.get("embeddings")
    
    # Flatten embedding if shape is (1, vector_length)
    if (
        embedding is not None 
        and isinstance(embedding, list) 
        and len(embedding) == 1 
        and isinstance(embedding[0], list)
    ):
        embedding = embedding[0]  # flatten (1,768) → (768,)
    
    if embedding is None:
        print("No embedding found, skipping.")
        return None
    
    # Build metadata
    properties = {
        "skills": skills_list,
        "job_desc": job_desc,
        "resume_file": os.path.basename(pdf_path),
        "hired": None,
        "timestamp": time.time()
    }
    unique_id = str(uuid.uuid4())
    
    # Add to ChromaDB collection (make sure 'collection' is initialized in your notebook/script)
    collection.add(
        ids=[unique_id],
        embeddings=[embedding],
        documents=[resume_text],
        metadatas=[properties]
    )
    print(f"Added resume with ID: {unique_id}")
    return unique_id


In [25]:
import json

properties = {
    "skills": json.dumps(skills_list),
    "job_desc": job_desc,
    "resume_file": os.path.basename(pdf_path),
    "hired": None,
    "timestamp": time.time()
}

NameError: name 'skills_list' is not defined

In [24]:
job_description = "Machine learning engineer. Required: Python, statistics, machine learning, artificial intelligence."
skills_required = ["python", "statistics", "machine learning", "artificial intelligence"]
resume_pdf_path = "/Users/sakshamfaujdar/Developer/NGIL/resume_reviewer/Saksham_resume.pdf"   # <-- update this path

resume_id = add_resume_from_pdf(resume_pdf_path, job_description, skills_required)

ValueError: Expected metadata value to be a str, int, float, bool, or None, got ['python', 'statistics', 'machine learning', 'artificial intelligence'] which is a list in add.